In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)

In [5]:
train_dtypes = pd.read_csv('../input/train8_dtypes.csv')
test_dtypes = pd.read_csv('../input/test8_dtypes.csv')
train_dtypes.index = train_dtypes['TransactionID']
del train_dtypes['TransactionID']
test_dtypes.index = test_dtypes['TransactionID']
del test_dtypes['TransactionID']
train_dtypes = train_dtypes.iloc[:,0]
test_dtypes = test_dtypes.iloc[:,0]
print(type(train_dtypes),type(test_dtypes))
train_dtypes = train_dtypes.to_dict()
test_dtypes = test_dtypes.to_dict()
gc.collect()

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


21

In [6]:
v_columns = ['V' + str(x) for x in range(1,340,1)]
c_columns = ['C' + str(x) for x in range(1,15,1)]

In [7]:
%%time
train = pd.read_csv('../input/train8.csv',dtype= train_dtypes,usecols = ['isFraud'] + v_columns + c_columns)
print("train finished")

train finished
CPU times: user 44.3 s, sys: 1.24 s, total: 45.5 s
Wall time: 45.5 s


In [8]:
%%time
test = pd.read_csv('../input/test8.csv',dtype= test_dtypes,usecols = v_columns + c_columns)
print("test finished")

test finished
CPU times: user 38.9 s, sys: 976 ms, total: 39.9 s
Wall time: 39.9 s


In [9]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 354) (506691, 353) (506691, 2)


In [10]:
two_columns = ['V12','V15','V19','V31','V33','V42','V43','V46','V47','V51','V52',\
               'V53','V57','V58','V61','V63','V64','V71','V72','V73','V74','V76',\
               'V79','V82','V84','V85','V92','V93','V108','V109','V110','V111',\
              'V112','V113','V114','V116','V117','V118','V119','V129','V194','V197',\
              'V260']
three_columns = ['V42','V51','V52','V82','V83','V85','V92','V93','V108','V110',\
                 'V111','V112','V113','V114']
greater_than_one = ['V39','V40','V80','V81','V146','V147','V148','V149','V153','V154','V155',\
                   'V156','V157','V158','V180','V184','V185','V186','V188','V189','V191',\
                   'V192','V193','V194','V195','V196','V198','V200','V201','V235','V245',\
                   'V247','V248','V249','V250','V251','V252','V253','V254','V255','V256',\
                   'V257','V258','V259','V261','V262']
greater_than_two = ['V37','V38','V44','V45','V77','V78','V86','V87','V187','V190','V199',\
                    'V242','V243','V244','V246','V140']
greater_than_one_C = ['C4','C7']
greater_than_two_C = ['C8','C10','C12']

In [11]:
train.head()

,isFraud,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [12]:
for i in tqdm(two_columns):
    train[i + '_is_2'] = train[i].apply(lambda x: 1 if x ==2 else 0 if np.isnan(x) == False else np.nan )
    test[i + '_is_2'] = test[i].apply(lambda x: 1 if x ==2 else 0 if np.isnan(x) == False else np.nan )    

100%|██████████| 43/43 [01:49<00:00,  2.53s/it]


In [13]:
for i in tqdm(three_columns):
    train[i + '_is_3'] = train[i].apply(lambda x: 1 if x ==3 else 0 if np.isnan(x) == False else np.nan )
    test[i + '_is_3'] = test[i].apply(lambda x: 1 if x ==3 else 0 if np.isnan(x) == False else np.nan )    

100%|██████████| 14/14 [00:34<00:00,  2.50s/it]


In [14]:
for i in tqdm(greater_than_one):
    train[i + '_greater_than_1'] = train[i].apply(lambda x: 1 if x > 1 else 0 if np.isnan(x) == False else np.nan )
    test[i + '_greater_than_1'] = test[i].apply(lambda x: 1 if x > 1 else 0 if np.isnan(x) == False else np.nan ) 

100%|██████████| 46/46 [02:00<00:00,  2.75s/it]


In [15]:
for i in tqdm(greater_than_two):
    train[i + '_greater_than_2'] = train[i].apply(lambda x: 1 if x > 2 else 0 if np.isnan(x) == False else np.nan )
    test[i + '_greater_than_2'] = test[i].apply(lambda x: 1 if x > 2 else 0 if np.isnan(x) == False else np.nan ) 

100%|██████████| 16/16 [00:43<00:00,  2.73s/it]


In [16]:
for i in tqdm(greater_than_one_C):
    train[i + '_greater_than_1'] = train[i].apply(lambda x: 1 if x > 1 else 0 if np.isnan(x) == False else np.nan )
    test[i + '_greater_than_1'] = test[i].apply(lambda x: 1 if x > 1 else 0 if np.isnan(x) == False else np.nan ) 

100%|██████████| 2/2 [00:04<00:00,  2.39s/it]


In [17]:
for i in tqdm(greater_than_two_C):
    train[i + '_greater_than_2'] = train[i].apply(lambda x: 1 if x > 2 else 0 if np.isnan(x) == False else np.nan )
    test[i + '_greater_than_2'] = test[i].apply(lambda x: 1 if x > 2 else 0 if np.isnan(x) == False else np.nan ) 

100%|██████████| 3/3 [00:07<00:00,  2.43s/it]


In [18]:
train['V139_greater_than_three'] = train['V139'].progress_apply(lambda x: 1 if x >= 2 else 0 if x < 3 else np.nan)
train['V140_greater_than_three'] = train['V140'].progress_apply(lambda x: 1 if x >= 3 else 0 if x < 3 else np.nan)
train['V151_isin'] = train['V151'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)
train['V152_isin'] = train['V152'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)
train['V168_isin'] = train['V168'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)
train['V170_isin'] = train['V170'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10,11,12] else 0 if np.isnan(x) == False else np.nan)
train['V171_isin'] = train['V171'].progress_apply(lambda x: 1 if x in [3,4,5,6,7,8,9,10,11,12,13] else 0 if np.isnan(x) == False else np.nan)
train['V176_isin'] = train['V176'].progress_apply(lambda x: 1 if x in [3,4,5,6,7,8,9,10,11] else 0 if np.isnan(x) == False else np.nan)
train['V218_isin'] = train['V218'].progress_apply(lambda x: 1 if x in [3,4,5,6,7,8,9,10,11,12,13,14] else 0 if np.isnan(x) == False else np.nan)
train['V228_isin'] = train['V228'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10] else 0 if np.isnan(x) == False else np.nan)
train['V229_isin'] = train['V229'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10] else 0 if np.isnan(x) == False else np.nan)
train['V230_isin'] = train['V230'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10] else 0 if np.isnan(x) == False else np.nan)
train['V231_isin'] = train['V231'].progress_apply(lambda x: 1 if x in [1,2,3,4,5,6,7,8,9,10,11] else 0 if np.isnan(x) == False else np.nan)
train['V232_isin'] = train['V232'].progress_apply(lambda x: 1 if x in [1,2,3,4,5,6,7,8,9,10,11] else 0 if np.isnan(x) == False else np.nan)
train['V233_isin'] = train['V233'].progress_apply(lambda x: 1 if x in [1,2,3,4,5,6,7,8,9,10,11,12] else 0 if np.isnan(x) == False else np.nan)
train['V234_isin'] = train['V234'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)

100%|██████████| 590540/590540 [00:00<00:00, 968226.06it/s] 


In [20]:
test['V139_greater_than_three'] = test['V139'].progress_apply(lambda x: 1 if x >= 2 else 0 if x < 3 else np.nan)
test['V140_greater_than_three'] = test['V140'].progress_apply(lambda x: 1 if x >= 3 else 0 if x < 3 else np.nan)
test['V151_isin'] = test['V151'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)
test['V152_isin'] = test['V152'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)
test['V168_isin'] = test['V168'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)
test['V170_isin'] = test['V170'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10,11,12] else 0 if np.isnan(x) == False else np.nan)
test['V171_isin'] = test['V171'].progress_apply(lambda x: 1 if x in [3,4,5,6,7,8,9,10,11,12,13] else 0 if np.isnan(x) == False else np.nan)
test['V176_isin'] = test['V176'].progress_apply(lambda x: 1 if x in [3,4,5,6,7,8,9,10,11] else 0 if np.isnan(x) == False else np.nan)
test['V218_isin'] = test['V218'].progress_apply(lambda x: 1 if x in [3,4,5,6,7,8,9,10,11,12,13,14] else 0 if np.isnan(x) == False else np.nan)
test['V228_isin'] = test['V228'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10] else 0 if np.isnan(x) == False else np.nan)
test['V229_isin'] = test['V229'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10] else 0 if np.isnan(x) == False else np.nan)
test['V230_isin'] = test['V230'].progress_apply(lambda x: 1 if x in [2,3,4,5,6,7,8,9,10] else 0 if np.isnan(x) == False else np.nan)
test['V231_isin'] = test['V231'].progress_apply(lambda x: 1 if x in [1,2,3,4,5,6,7,8,9,10,11] else 0 if np.isnan(x) == False else np.nan)
test['V232_isin'] = test['V232'].progress_apply(lambda x: 1 if x in [1,2,3,4,5,6,7,8,9,10,11] else 0 if np.isnan(x) == False else np.nan)
test['V233_isin'] = test['V233'].progress_apply(lambda x: 1 if x in [1,2,3,4,5,6,7,8,9,10,11,12] else 0 if np.isnan(x) == False else np.nan)
test['V234_isin'] = test['V234'].progress_apply(lambda x: 1 if x in [2,3,4,5,6] else 0 if np.isnan(x) == False else np.nan)

100%|██████████| 506691/506691 [00:01<00:00, 292586.50it/s]


In [21]:
print(train.shape,test.shape)

(590540, 494) (506691, 493)


In [29]:
test.iloc[:,353:].head()

,V12_is_2,V15_is_2,V19_is_2,V31_is_2,V33_is_2,V42_is_2,V43_is_2,V46_is_2,V47_is_2,V51_is_2,V52_is_2,V53_is_2,V57_is_2,V58_is_2,V61_is_2,V63_is_2,V64_is_2,V71_is_2,V72_is_2,V73_is_2,V74_is_2,V76_is_2,V79_is_2,V82_is_2,V84_is_2,V85_is_2,V92_is_2,V93_is_2,V108_is_2,V109_is_2,V110_is_2,V111_is_2,V112_is_2,V113_is_2,V114_is_2,V116_is_2,V117_is_2,V118_is_2,V119_is_2,V129_is_2,V194_is_2,V197_is_2,V260_is_2,V42_is_3,V51_is_3,V52_is_3,V82_is_3,V83_is_3,V85_is_3,V92_is_3,V93_is_3,V108_is_3,V110_is_3,V111_is_3,V112_is_3,V113_is_3,V114_is_3,V39_greater_than_1,V40_greater_than_1,V80_greater_than_1,V81_greater_than_1,V146_greater_than_1,V147_greater_than_1,V148_greater_than_1,V149_greater_than_1,V153_greater_than_1,V154_greater_than_1,V155_greater_than_1,V156_greater_than_1,V157_greater_than_1,V158_greater_than_1,V180_greater_than_1,V184_greater_than_1,V185_greater_than_1,V186_greater_than_1,V188_greater_than_1,V189_greater_than_1,V191_greater_than_1,V192_greater_than_1,V193_greater_than_1,V194_greater_than_1,V195_greater_than_1,V196_greater_than_1,V198_greater_than_1,V200_greater_than_1,V201_greater_than_1,V235_greater_than_1,V245_greater_than_1,V247_greater_than_1,V248_greater_than_1,V249_greater_than_1,V250_greater_than_1,V251_greater_than_1,V252_greater_than_1,V253_greater_than_1,V254_greater_than_1,V255_greater_than_1,V256_greater_than_1,V257_greater_than_1,V258_greater_than_1,V259_greater_than_1,V261_greater_than_1,V262_greater_than_1,V37_greater_than_2,V38_greater_than_2,V44_greater_than_2,V45_greater_than_2,V77_greater_than_2,V78_greater_than_2,V86_greater_than_2,V87_greater_than_2,V187_greater_than_2,V190_greater_than_2,V199_greater_than_2,V242_greater_than_2,V243_greater_than_2,V244_greater_than_2,V246_greater_than_2,V140_greater_than_2,C4_greater_than_1,C7_greater_than_1,C8_greater_than_2,C10_greater_than_2,C12_greater_than_2,V139_greater_than_three,V140_greater_than_three,V151_isin,V152_isin,V168_isin,V170_isin,V171_isin,V176_isin,V218_isin,V228_isin,V229_isin,V230_isin,V231_isin,V232_isin,V233_isin,V234_isin
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0

In [30]:
%%time
train.iloc[:,354:].to_csv('train_Vcolumns.csv',index=False)
test.iloc[:,353:].to_csv('test_Vcolumns.csv',index=False)

CPU times: user 1min 20s, sys: 1.4 s, total: 1min 22s
Wall time: 1min 22s


In [ ]:
train.groupby('isFraud')['V203'].max()

In [ ]:
plt.hist(train['V203'],bins=1000)
plt.show()

In [ ]:
train['V233'].head()

In [ ]:
train.groupby('C10')['isFraud'].mean().head(20)

In [ ]:
train.groupby('V233')['isFraud'].count().head(30)

# Train data is subsampled

In [ ]:
train_0 = train[train['isFraud'] == 0].sample(frac = 0.2)
train_1 = train[train['isFraud'] == 1]
del train
gc.collect()
print(train_0.shape,train_1.shape)
train = pd.concat([train_0,train_1],axis=0)
train = train.sample(frac=1.0)
train = train.reset_index(drop=True)
print(train.shape,test.shape)
del train_0,train_1
gc.collect()

# Training

In [ ]:
train.drop(['TransactionID','TransactionDT','dayofyear','day'],axis=1,inplace=True)
test.drop(['TransactionID','TransactionDT','dayofyear','day'],axis=1,inplace=True)
gc.collect()

In [ ]:
y = train['isFraud']
del train['isFraud']

In [ ]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.04,
                    'num_leaves': 2**8,
                    'max_depth':12,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':500,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 51,
                    'early_stopping_rounds':100, 
                } 

In [ ]:
N = 5
kf = KFold(n_splits=N)

importance = pd.DataFrame(np.zeros((train.shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train.columns)
scores = []
y_pred = np.zeros(test.shape[0])
oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(kf.split(train, y), 1):
    print('Fold {}'.format(fold))
          
    trn_data = lgb.Dataset(train.iloc[trn_idx, :].values, label=y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train.iloc[val_idx, :].values, label=y.iloc[val_idx].values)   
    
    clf = lgb.train(lgb_params, trn_data ,valid_sets=[trn_data, val_data], verbose_eval=100)

    predictions = clf.predict(train.iloc[val_idx, :].values) 
    importance.iloc[:, fold - 1] = clf.feature_importance()
    oof[val_idx] = predictions

    score = roc_auc_score(y.iloc[val_idx].values, predictions)
    scores.append(score)
    print('Fold {} ROC AUC Score {}\n'.format(fold, score))

    y_pred += clf.predict(test) / N
    
    del trn_data, val_data, predictions
    gc.collect()
    
print('Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))

In [ ]:
print(roc_auc_score(y,oof))

In [ ]:
importance['avg'] = importance.mean(axis=1)

In [ ]:
importance = importance.sort_values(by='avg',ascending=False)

In [ ]:
importance = importance.reset_index(drop=False)

In [ ]:
importance.head()

In [ ]:
importance.to_csv('../importances/importances16.csv',index=False)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission['isFraud'] = y_pred

In [ ]:
sample_submission.head()

In [ ]:
plt.hist(sample_submission['isFraud'],bins=200)
plt.show()

In [ ]:
sample_submission.to_csv('../predictions/pred16.csv',index=False)

In [ ]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred16.csv -m "pred16"